In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

%cd ../../src

/media/premium/common-biscuit/main/planogram_biscuit/src


In [3]:
from scoring.Locator import Locator
locator = Locator() 


In [2]:
import os
import sys
import json

In [4]:
 
pred_dir = '/media/premium/common-biscuit/main/planogram_biscuit/data/output/image_annotations/integrated/op_annotations_rack'


In [5]:
predictions = os.listdir(pred_dir)

In [8]:
predictions

['PHOTO-2021-07-20-20-32-07 3.json',
 'PHOTO-2021-07-20-20-32-07.json',
 'PHOTO-2021-07-21-07-45-43 2.json',
 'PHOTO-2021-07-21-07-45-43.json',
 'PHOTO-2021-07-21-07-56-20.json',
 'PHOTO-2021-07-21-08-05-38 2.json',
 'PHOTO-2021-07-21-08-05-38.json',
 'PHOTO-2021-07-21-08-13-01.json',
 'PHOTO-2021-07-21-08-13-02.json',
 'PHOTO-2021-07-21-08-21-00 2.json',
 'PHOTO-2021-07-21-08-21-00.json',
 'PHOTO-2021-07-21-08-35-00 2.json',
 'PHOTO-2021-07-21-08-35-00.json',
 'PHOTO-2021-07-21-08-38-26.json',
 'PHOTO-2021-07-21-08-59-06.json',
 'PHOTO-2021-07-21-09-02-19.json',
 'PHOTO-2021-07-21-09-11-07.json',
 'PHOTO-2021-07-21-09-17-10 2.json',
 'PHOTO-2021-07-21-09-17-10.json',
 'PHOTO-2021-07-21-09-19-00.json',
 'PHOTO-2021-07-21-09-24-05 2.json',
 'PHOTO-2021-07-21-09-43-14.json',
 'PHOTO-2021-07-21-09-57-52.json',
 'PHOTO-2021-07-21-09-58-17.json',
 'PHOTO-2021-07-21-10-32-47.json',
 'PHOTO-2021-07-21-10-43-25.json',
 'PHOTO-2021-07-21-10-43-46.json',
 'PHOTO-2021-07-21-10-52-44.json',
 'PHOT

In [7]:
with open(os.path.join(pred_dir,predictions[0])) as file:
            data = json.loads(file.read())

In [9]:
rack_rows, packets,complete_rack = locator.get_rows_packets_predictions(data)


array([(326, 35, 823, 1376)], dtype=object)

In [23]:
packets

array([(530, 1460, 628, 1597), (362, 589, 460, 667),
       (577, 1150, 675, 1324), (462, 11, 606, 168), (334, 560, 458, 658),
       (331, 603, 439, 679), (607, 107, 735, 208), (415, 0, 544, 148),
       (552, 617, 661, 675), (550, 820, 628, 894), (413, 1182, 604, 1438),
       (524, 1, 721, 221), (378, 799, 525, 962), (463, 277, 562, 395),
       (499, 819, 610, 910), (454, 1410, 618, 1599), (379, 0, 486, 148),
       (601, 297, 684, 390), (577, 590, 718, 659), (416, 1408, 521, 1582),
       (495, 266, 617, 392), (624, 1132, 714, 1324), (565, 772, 667, 882),
       (619, 1398, 704, 1511), (520, 1231, 676, 1416),
       (366, 1221, 547, 1471), (656, 1363, 747, 1472),
       (548, 969, 641, 1134), (337, 805, 486, 997),
       (562, 1445, 673, 1554), (318, 274, 526, 488),
       (581, 946, 690, 1091), (429, 811, 570, 934), (486, 995, 585, 1179),
       (422, 542, 590, 846), (377, 1040, 520, 1230)], dtype=object)

In [11]:
#sorting the rack rows according to the y coordinates
rack_rows = locator.get_sorted_rows(rack_rows)
rack_rows

array([(471, 0, 790, 108), (336, 42, 780, 402), (519, 436, 778, 565),
       (527, 559, 769, 697), (350, 677, 769, 955), (448, 881, 745, 1217),
       (496, 1062, 753, 1499), (546, 1299, 761, 1599)], dtype=object)

In [13]:
average_rackrow_height=locator.get_average_height(rack_rows)
average_rackrow_height

282.57142857142856

In [14]:
average_rackrow_width=locator.get_average_width(rack_rows)
average_rackrow_width

306.5

In [17]:
imputed_missing_rackrows=locator.find_missing_row_between(rack_rows,average_rackrow_height)

In [18]:
#adds missing rackrows to images and sorts them
imputed_missing_rackrows

array([(471, 0, 790, 108), (336, 42, 780, 402), (519, 436, 778, 565),
       (527, 559, 769, 697), (350, 677, 769, 955), (448, 881, 745, 1217),
       (496, 1062, 753, 1499), (546, 1299, 761, 1599)], dtype=object)

In [19]:
def locate_packets(pred_dir, predictions):
    
    location_dict = {}
    for prediction in predictions:
        
        with open(os.path.join(pred_dir,prediction)) as file:
            data = json.loads(file.read())
        
        updated_packet_positions, rack_rows = find_packet_positions(data)
        location_dict[prediction] = [np.array(updated_packet_positions), np.array(rack_rows)]
 
    return location_dict

In [21]:
def find_packet_positions(data):
    rack_rows, packets, complete_rack = locator.get_rows_packets_predictions(data)
    rack_rows = locator.get_sorted_rows(rack_rows)
    avg_row_height = locator.get_average_height(rack_rows)
    
    rack_rows = locator.find_missing_row_between(rack_rows, avg_row_height, 
                                                 thresholds['missing_row_thresh'],
                                                 thresholds['missing_mid_row_thresh'])
    
    # first_row = locator.find_missing_first_row_modified(rack_rows, avg_row_height, packets, complete_rack,
    #                                            thresholds['missing_top_row_packet_overlap_thresh'], 
    #                                            thresholds['top_row_additional_pixels'], 
    #                                            thresholds['top_row_additional_thresh_pixels'])
    
    # if first_row is not None:
    #     rack_rows = np.insert(rack_rows, 0, first_row)

    # Check which packet is in which row.
    packet_positions = locator.check_packet_row(rack_rows, packets, 
                                                thresholds['packet_row_overlap_thresh'],
                                                thresholds['packet_top_row_overlap_thresh'])
    
    # Fill blank bboxes.
    updated_packet_positions = locator.fill_blank_bbox(packets, packet_positions, rack_rows,
                                                       thresholds['empty_row_bbox_thresh'],
                                                       thresholds['blank_middle_bbox_thresh'],
                                                       thresholds['blank_start_bbox_thresh'],
                                                       thresholds['blank_end_bbox_thresh'])
    
    #Find Blank boxes that are overlapping with actual boxes.
    overlap_blank = locator.find_overlapping_blank_bbox(updated_packet_positions, packets, 
                                                        iou_thresh=thresholds['blank_predicted_overlap_iou'])
                        
    # Remove blank bounding boxes that overlap other boxes.
    updated_packet_positions = locator.remove_overlapping_bbox(overlap_blank, updated_packet_positions)
    return updated_packet_positions, rack_rows

In [22]:
location_dict=locate_packets(pred_dir, predictions)

NameError: name 'thresholds' is not defined

In [27]:
def find_missing_first_row_modified(self, rack_rows, avg_row_height, packets, complete_rack, overlap_thresh=0.7, add_pixels=25, thresh_pixels=50):
        if (complete_rack[0].get_bbox()[1] + rack_rows[0].get_bbox()[1]) > avg_row_height:
            try:
                x1 = np.max(np.array([row.get_bbox()[0] for row in rack_rows])) 
            except Exception as e:
                print("No rack rows present")
                return None
            y1 = rack_rows[0].get_bbox()[1] - avg_row_height
            if y1 > thresh_pixels:
                y1 = y1 - add_pixels
            x2 = np.max(np.array([row.get_bbox()[2] for row in rack_rows]))
            y2 = rack_rows[0].get_bbox()[1]
            row = Row(x1, y1, x2, y2)
            # check if y1 coordinate becoame -ve.
            if y1 < 0:
                return None
            
            # check if any packets in this row.
            for packet in packets:
                box_area = Evaluator._getArea(packet.get_bbox())
                intersection_area = Evaluator._getIntersectionArea(row.get_bbox(), packet.get_bbox())
                overlap_perc = intersection_area/box_area

                if overlap_perc > overlap_thresh:
                    return row
            return None
        else:
            return None

In [28]:
rows=locator.find_missing_first_row_modified(rack_rows,average_rackrow_height,packets,complete_rack)
print(rows)

None


In [31]:
def check_packet_row(self, rack_rows, packets, overlap_thresh=0.7, top_overlap_thresh=0.7):
        
        packet_positions = np.empty((rack_rows.shape[0],), dtype=object)
    
        for i,v in enumerate(packet_positions): packet_positions[i]=[]

        for packet in packets:
            row_num = self.packet_overlap(rack_rows, packet, overlap_thresh, top_overlap_thresh)
            if row_num != -1:
                packet_positions[row_num].append(packet)

        packet_positions = self.get_sorted_columns(packet_positions)
        
        return packet_positions
    

In [33]:
packet_positions=locator.check_packet_row(rack_rows,packets)

In [34]:
packet_positions

array([array([], dtype=float64),
       array([(379, 0, 486, 148), (415, 0, 544, 148), (462, 11, 606, 168),
       (463, 277, 562, 395), (495, 266, 617, 392), (524, 1, 721, 221),
       (601, 297, 684, 390), (607, 107, 735, 208)], dtype=object),
       array([], dtype=float64),
       array([(552, 617, 661, 675), (577, 590, 718, 659)], dtype=object),
       array([(337, 805, 486, 997), (378, 799, 525, 962), (429, 811, 570, 934),
       (499, 819, 610, 910), (550, 820, 628, 894), (565, 772, 667, 882)],
      dtype=object),
       array([(486, 995, 585, 1179), (548, 969, 641, 1134),
       (581, 946, 690, 1091)], dtype=object),
       array([(520, 1231, 676, 1416), (577, 1150, 675, 1324),
       (619, 1398, 704, 1511), (624, 1132, 714, 1324),
       (656, 1363, 747, 1472)], dtype=object),
       array([(530, 1460, 628, 1597), (562, 1445, 673, 1554)], dtype=object)],
      dtype=object)